# Diagram Tests

## Setup

### Imports

In [1]:
# Type imports
from typing import Any, Dict, List, Tuple

# Standard Libraries
from pathlib import Path
from collections import Counter, defaultdict
import math
from math import sqrt, pi, sin, cos, tan, radians
from statistics import mean
from itertools import zip_longest

# Shared Packages
import pandas as pd
import numpy as np
import xlwings as xw

import PySimpleGUI as sg

import pygraphviz as pgv
import networkx as nx


In [2]:
from types_and_classes import *

### Matplotlib settings

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Arc
from matplotlib_inline.backend_inline import set_matplotlib_formats
import matplotlib.font_manager as font_manager

#plt.ion()

#set_matplotlib_formats('svg')

In [4]:
%matplotlib inline

# Example Graph Data

In [5]:
folder=Path.cwd()
name='test_graph'
full_dot_name = name + '.gv'
full_png_name = name + '.png'
full_svg_name = name + '.svg'
dot_path = folder / full_dot_name
png_path = folder / full_png_name
svg_path = folder / full_svg_name


In [6]:
structure_def = [
    {
        'struct_id': 'GTV',
        'roi': 38,
        'structure_type': 'GTV',
        'structure_code': 'GTVp',
        'structure_code_meaning': 'Primary Gross Tumor Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (255, 0, 0),
        'volume': 8.03,
        'length': 2.6,
        'sup_slice': -0.4,
        'inf_slice': -3,
        'center_of_mass': (-5.36,  9.71, -1.63)
        },{
        'struct_id': 'CTV',
        'roi': 24,
        'structure_type': 'GTV',
        'structure_code': 'ITV',
        'structure_code_meaning': 'Internal Target Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (255, 255, 0),
        'volume': 34.45,
        'length': 3.6,
        'sup_slice': 0,
        'inf_slice': -3.6,
        'center_of_mass': (-5.34,  9.79, -1.61)
        },{
        'struct_id': 'PTV',
        'roi': 30,
        'structure_type': 'PTV',
        'structure_code': 'PTVp',
        'structure_code_meaning': 'Primary Planning Target Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (0, 255, 255),
        'volume': 74.649,
        'length': 4.6,
        'sup_slice': 0.6,
        'inf_slice': -4.0,
        'center_of_mass': (-5.36,  9.78, -1.59)
        },{
        'struct_id': 'eval PTV',
        'roi': 41,
        'structure_type': 'PTV',
        'structure_code': 'PTVp',
        'structure_code_meaning': 'Primary Planning Target Volume',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (0, 255, 255),
        'volume': 74.649,
        'length': 4.6,
        'sup_slice': 0.6,
        'inf_slice': -4.0,
        'center_of_mass': (-5.36,  9.78, -1.59)
        },{
        'struct_id': 'BODY',
        'roi': 1,
        'structure_type': 'EXTERNAL',
        'structure_code': 'BODY',
        'structure_code_meaning': 'Body',
        'structure_code_scheme': '99VMS_STRUCTCODE',
        'color': (0, 255, 0),
        'volume': 28951.626,
        'length': 33.8,
        'sup_slice': 10.6,
        'inf_slice': -23.2,
        'center_of_mass': (-0.95,  9.73, -6.76)
        },{
        'struct_id': 'Lung L',
        'roi': 26,
        'structure_type': 'ORGAN',
        'structure_code': '7310',
        'structure_code_meaning': 'Left lung',
        'structure_code_scheme': 'FMA',
        'color': (224, 255, 255),
        'volume': 1776,
        'length': 24.2,
        'sup_slice': 5.8,
        'inf_slice': -18.4,
        'center_of_mass': (7.08, 10.61, -6.28)
        },{
        'struct_id': 'Lung R',
        'roi': 27,
        'structure_type': 'ORGAN',
        'structure_code': '7309',
        'structure_code_meaning': 'Right lung',
        'structure_code_scheme': 'FMA',
        'color': (255, 218, 185),
        'volume': 2556.676,
        'length': 23.8,
        'sup_slice': 6.6,
        'inf_slice': -17.2,
        'center_of_mass': (-8.09,  8.77, -5.57)
        },{
        'struct_id': 'Lung B',
        'roi': 25,
        'structure_type': 'ORGAN',
        'structure_code': '68877',
        'structure_code_meaning': 'Pair of lungs',
        'structure_code_scheme': 'FMA',
        'color': (218, 165, 32),
        'volume': 4332.676,
        'length': 25,
        'sup_slice': 6.6,
        'inf_slice': -18.4,
        'center_of_mass': (-1.87,  9.52, -5.86)
        },{
        'struct_id': 'Skin',
        'roi': 2,
        'structure_type': 'ORGAN',
        'structure_code': '7163',
        'structure_code_meaning': 'Skin',
        'structure_code_scheme': 'FMA',
        'color': (240, 255, 240),
        'volume': 1726.808,
        'length': 33.8,
        'sup_slice': 10.6,
        'inf_slice': -23.2,
        'center_of_mass': (-0.95,  9.73, -6.76)
        }
    ]

structure_list = []

for node_dict in structure_def:
    structure_list.append(Structure(**node_dict))


In [16]:
edge_def = [
    {'structures': (1, 2), 'relationship': 'CONFINES'},
    {'structures': (1, 25), 'relationship': 'CONTAINS'},
    {'structures': (1, 27), 'relationship': 'CONTAINS'},
    {'structures': (1, 26), 'relationship': 'CONTAINS'},
    {'structures': (1, 30), 'relationship': 'CONTAINS'},
    {'structures': (1, 41), 'relationship': 'CONTAINS'},
    {'structures': (1, 24), 'relationship': 'CONTAINS'},
    {'structures': (1, 38), 'relationship': 'CONTAINS'},
    {'structures': (2, 25), 'relationship': 'DISJOINT'},
    {'structures': (2, 27), 'relationship': 'DISJOINT'},
    {'structures': (2, 26), 'relationship': 'DISJOINT'},
    {'structures': (2, 30), 'relationship': 'SURROUNDS'},
    {'structures': (2, 41), 'relationship': 'SURROUNDS'},
    {'structures': (2, 24), 'relationship': 'SURROUNDS'},
    {'structures': (2, 38), 'relationship': 'SURROUNDS'},
    {'structures': (25, 27), 'relationship': 'CONTAINS'},
    {'structures': (25, 26), 'relationship': 'CONTAINS'},
    {'structures': (25, 30), 'relationship': 'OVERLAPS'},
    {'structures': (25, 41), 'relationship': 'CONTAINS'},
    {'structures': (25, 24), 'relationship': 'CONTAINS'},
    {'structures': (25, 38), 'relationship': 'CONTAINS'},
    {'structures': (27, 26), 'relationship': 'DISJOINT'},
    {'structures': (27, 30), 'relationship': 'DISJOINT'},
    {'structures': (27, 41), 'relationship': 'DISJOINT'},
    {'structures': (27, 24), 'relationship': 'DISJOINT'},
    {'structures': (27, 38), 'relationship': 'DISJOINT'},
    {'structures': (26, 30), 'relationship': 'OVERLAPS'},
    {'structures': (26, 41), 'relationship': 'INCORPORATES'},
    {'structures': (26, 24), 'relationship': 'CONTAINS'},
    {'structures': (26, 38), 'relationship': 'CONTAINS'},
    {'structures': (30, 41), 'relationship': 'INCORPORATES'},
    {'structures': (30, 24), 'relationship': 'CONTAINS'},
    {'structures': (30, 38), 'relationship': 'CONTAINS'},
    {'structures': (41, 24), 'relationship': 'CONTAINS'},
    {'structures': (41, 38), 'relationship': 'CONTAINS'},
    {'structures': (24, 38), 'relationship': 'CONTAINS'},
    ]

In [21]:
relationship_list = []

for edge_dict in edge_def:
    edge = list(edge_dict['structures']) + [Relationship(**edge_dict)]
    relationship_list.append(edge)
relationships = pd.DataFrame(relationship_list)
relationships.columns = ['ROI_1', 'ROI_2', 'Relationship']
relationships

,ROI_1,ROI_2,Relationship
0,1,2,<types_and_classes.Relationship object at 0x00...
1,1,25,<types_and_classes.Relationship object at 0x00...
2,1,27,<types_and_classes.Relationship object at 0x00...
3,1,26,<types_and_classes.Relationship object at 0x00...
4,1,30,<types_and_classes.Relationship object at 0x00...
5,1,41,<types_and_classes.Relationship object at 0x00...
6,1,24,<types_and_classes.Relationship object at 0x00...
7,1,38,<types_and_classes.Relationship object at 0x00...
8,2,25,<types_and_classes.Relationship object at 0x00...
9,2,27,<types_and_classes.Relationship object at 0x00...


In [ ]:

structures: StructurePair = tuple([38, 24])
r = Relationship(structures)

In [ ]:
r.metric

In [ ]:
g = StructureDiagram()
g.add_structure_nodes(structure_list)
g.display_graph.layout(prog='dot')

In [ ]:
g.display_graph.draw(png_path)
g.display_graph.draw(svg_path)
img = mpimg.imread(png_path)
fig, ax = plt.subplots()
ax.imshow(img)
ax.axis('off')
plt.show()

In [ ]:
#from graph_matrix import from_adjacency_matrix
#from multi_graph import plot_multi_edge

## Save and display file with PyGraphVis

In [ ]:
G = pgv.AGraph(
    labelloc='t',
    clusterrank='none',
    bgcolor='#555555',
    fontname = 'Helvetica,Arial,sans-serif',
    fontsize = 16,
    fontcolor = 'white',
    label = r'Example Structure Relations\n'
    )

In [ ]:
G.node_attr.update(
    fixedsize = 'shape',
    width = 1,
    height = 0.6,
    fontname = 'Helvetica-Bold',
    fontsize = 12,
    labelloc = 'c',
    penwidth = 3,
    style='filled',
    fontcolor = 'black'
    )


In [ ]:
G.edge_attr.update(
    color = '#e27dd6ff',
    penwidth = 3,
    style = 'solid',
    arrowhead = 'none',
    arrowtail = 'none',
    labelfloat=False,
    labelfontname = 'Times',
    fontcolor = '#55AAFF',
    fontsize = '10'
    )

In [ ]:
node_props = {
    'GTV': {'class': "Target",
            'shape': "pentagon",
            'color': "#ff0000",
            'label': "GTV"},
    'CTV': {'class': 'Target',
            'shape': 'hexagon',
            'color': '#FFFF00',
            'label': 'CTV'},
    'PTV': {'class': 'Target',
            'shape': 'octagon',
            'color': '#00FFFF',
            'label': 'PTV'},
    'eval_PTV': {'class':'Target',
                'shape': 'octagon',
                'color': '#00FFFF',
                'label': 'eval PTV'},
    'Lung_L': {'class': 'OAR',
               'shape': 'rectangle',
               'style': 'rounded, filled',
               'color': '#E0FFFF',
               'label': 'Lung L'},
    'Lung_R': {'class': 'OAR',
               'shape': 'rectangle',
               'style': 'rounded, filled',
               'color': '#FFDAB9',
               'label': 'Lung R'},
    'Lung_B': {'class': 'OAR',
               'shape': 'rectangle',
               'style': 'rounded, filled',
               'color': '#DAA532',
               'label': 'Lung B'},
    'Skin': {'class':'OAR',
             'shape': 'rectangle',
             'style': 'rounded, filled',
             'color': '#F0FFF0',
             'label': 'Skin'},
    'Body': {'class':'External',
             'shape': 'oval',
             'color': '#00ff00',
             'label': 'Body'}}
for node_id, values in node_props.items():
    G.add_node(node_id, **values)

G.add_subgraph(['GTV', 'CTV', 'PTV', 'eval_PTV'], name='Targets', cluster=True)
G.add_subgraph(['Lung_L', 'Lung_R', 'Lung_B', 'Skin'], name='OAR', cluster=True)
G.add_subgraph(['Body'], name='External', cluster=True)

In [ ]:
edge_props = {
    ('GTV', 'CTV'): {'class': 'Equals',
                     'style': 'tapered',
                     'penwidth': 5,
                     'dir': 'both'},
    ('GTV', 'PTV'): {'class': 'Contain',
                     'style': 'dotted',
                     'penwidth': 1},
    ('GTV', 'eval_PTV'): {'class': 'Contain',
                          'style': 'dotted',
                          'penwidth': 1},
    ('GTV', 'Skin'): {'class': 'Island',
                      'style': 'dotted',
                      'headlabel': 'Embeds',
                      'taillabel': 'Surrounds'},
    ('GTV', 'Lung_L'): {'class': 'Contain',
                        'style': 'dotted',
                        'penwidth': 1},
    ('GTV', 'Lung_B'): {'class': 'Contain',
                        'style': 'dotted',
                        'penwidth': 1},
    ('CTV', 'PTV'): {'class': 'Contain',
                     'style': 'tapered',
                     'penwidth': 5,
                     'dir': 'forward'},
    ('CTV', 'Skin'): {'class': 'Island',
                      'style': 'dotted',
                      'headlabel': 'Embeds',
                      'taillabel': 'Surrounds'},
    ('CTV', 'Lung_L'): {'class': 'Contain',
                        'style': 'dotted',},
    ('PTV', 'eval_PTV'): {'class': 'Group',
                          'style': 'solid',
                          'headlabel': 'Incorporates',
                          'taillabel': 'Partitions'},
    ('PTV', 'Lung_L'): {'label': 'Overlaps',
                        'style': 'bold'},
    ('eval_PTV', 'Lung_L'): {'class': 'Group',
                             'style': 'solid',
                             'headlabel': 'Partitions',
                             'taillabel': 'Incorporates'},
    ('Lung_L', 'Lung_B'): {'class': 'Contain',
                           'style': 'tapered',
                           'penwidth': 5,
                           'dir': 'back'},
    ('Lung_R', 'Lung_B'): {'class': 'Contain',
                           'style': 'tapered',
                           'penwidth': 5,
                           'dir': 'back'},
    ('Body', 'Skin'): {'class': 'Cut-out',
                       'style': 'dashed',
                       'headlabel': 'Partitions',
                       'taillabel': 'Incorporates'},
    ('Body', 'Lung_L'): {'class': 'Contain',
                         'style': 'tapered',
                         'penwidth': 5,
                         'dir': 'forward'},
    ('Body', 'Lung_R'): {'class': 'Contain',
                         'style': 'tapered',
                         'penwidth': 5,
                         'dir': 'forward'},
    ('Body', 'Lung_B'): {'class': 'Contain',
                         'style': 'tapered',
                         'penwidth': 5,
                         'dir': 'forward'},
}
for edge_id, values in edge_props.items():
    G.add_edge(edge_id[0], edge_id[1], **values)


In [ ]:
G.layout(prog='dot')

In [ ]:
folder=Path.cwd()
name='test_graph'
full_dot_name = name + '.gv'
full_png_name = name + '.png'
full_svg_name = name + '.svg'
dot_path = folder / full_dot_name
png_path = folder / full_png_name
svg_path = folder / full_svg_name

G.draw(png_path)
G.draw(svg_path)
img = mpimg.imread(png_path)
fig, ax = plt.subplots()
ax.imshow(img)
ax.axis('off')
plt.show()

In [ ]:
print(G.to_string())

```json
#CTV -- eval_PTV [label="Contain"]
#CTV -- Lung_B [label="Contain"]
#PTV -- Skin [label="Island"]
#PTV -- Lung_B [label="Overlaps"]
#eval_PTV -- Skin [label="Island"]
#eval_PTV -- Lung_B [label="Contain"]
#Body -- GTV [label="Contain"]
#Body -- CTV [label="Contain"]
#Body -- PTV [label="Contain"]
#Body -- eval_PTV [label="Contain"]
```

```json
GTV [tooltip = "ID: GTV\nROI: 38\n\nVolumeType: GTV\nDicomType: GTV\nCode: GTVp\nLabel: Primary Gross Tumor Volume\nScheme: 99VMS_STRUCTCODE\nVolume: 8.0\nEq Sp Diam: 2.5\nLength: 2.6\nRange: (-0.4, -3.0)"]
CTV [tooltip = "ID: CTV\nROI: 24\nName: Internal Target Volume\nVolumeType: CTV\nDicomType: CTV\nCode: ITV\nLabel: Internal Target Volume\nScheme: 99VMS_STRUCTCODE\nVolume: 34.4\nEq Sp Diam: 4.0\n\nLength: 3.6\nRange: (0.0, -3.6)"]
PTV [tooltip = "ID: PTV\nROI: 30\nName: PTV defined by Radiation Oncologist\nVolumeType: PTV\nDicomType: PTV\nCode: PTVp\nLabel: Primary Planning Target Volume\nScheme: 99VMS_STRUCTCODE\nVolume: 74.6\nEq Sp Diam: 5.2\n\nLength: 4.6\nRange: (0.6, -4.0)"]
Body [tooltip = "ID: Body\nROI: 1\n\nVolumeType: BODY\nDicomType: EXTERNAL\nCode: BODY\nLabel: Body\nScheme: 99VMS_STRUCTCODE\nVolume: 28,951.6\nEq Sp Diam: 38.1\nRadius: 16.5\nLength: 33.8\nRange: (10.6, -23.2)"]
eval_PTV [tooltip = "ID: eval_PTV\nROI: 41\nName: PTV High Risk for dvh\nVolumeType: PTV\nDicomType: PTV\nCode: PTVp\nLabel: Primary Planning Target Volume\nScheme: 99VMS_STRUCTCODE\nVolume: 630.6\nEq Sp Diam: 10.6\n\nLength: 9.8\nRange: (1.4, -8.4)"]
Lung_L [tooltip = "ID: Lung_L\nROI: 26\nName: Left Lung\nVolumeType: Organ\nDicomType: ORGAN\nCode: 7310\nLabel: Left lung\nScheme: FMA\nVolume: 1,776.0\nEq Sp Diam: 15.0\n\nLength: 24.2\nRange: (5.8, -18.4)"]
Lung_R [tooltip = "ID: Lung_R\nROI: 27\nName: Right Lung\nVolumeType: Organ\nDicomType: ORGAN\nCode: 7309\nLabel: Right lung\nScheme: FMA\nVolume: 2,556.7\nEq Sp Diam: 17.0\n\nLength: 23.8\nRange: (6.6, -17.2)"]
Lung_B [tooltip = "ID: Lung_B\nROI: 25\nName: Both Lungs\nVolumeType: Organ\nDicomType: ORGAN\nCode: 68877\nLabel: Pair of lungs\nScheme: FMA\nVolume: 4,332.7\nEq Sp Diam: 20.2\n\nLength: 25.0\nRange: (6.6, -18.4)"]
Skin [tooltip = "ID: Skin\n\nName: Skin\nVolumeType: Organ\nDicomType: Organ\nCode: 7163\nLabel: Skin\nScheme: FMA\nVolume: 1,072.3\n\n\nLength: 33.8\nRange: (10.6, -23.2)"]
```

```json
graph StructureSet {
    graph [labelloc = t, 
          clusterrank=none, 
          bgcolor="#555555",
          fontname = "Helvetica,Arial,sans-serif", 
          fontsize = 16, 
          fontcolor = "white"
          ]
    node [fixedsize = "shape",
          width = 1,
          height = .6,
          fontname = "Helvetica-Bold",
          fontsize = 12,
          labelloc = "c",
          penwidth = 3,
          style="filled",
          fontcolor = "black"
          ]
    edge [color = "#e27dd6ff",
          penwidth = 3,
          style = "solid",
          arrowhead = "none",
          arrowtail = "none",
          labelfloat=false,
          labelfontname = "Times",
          fontcolor = "#55AAFF",
          fontsize = "10"
          ]
    subgraph Targets {
        cluster = true;
        GTV [class = "Target", shape = "pentagon", color = "#ff0000", label = "GTV"]
        CTV [class= "Target", shape = "hexagon", color = "#FFFF00" label = "CTV"]
        PTV [class= "Target", shape = "octagon", color = "#00FFFF", label = "PTV"]
        eval_PTV [class= "Target", shape = "octagon" color = "#00FFFF" label = "eval PTV"]
        }
    subgraph OAR {cluster = true; 
        node [shape = "rectangle" style="rounded, filled"]        
        Lung_L [class= "OAR", color = "#E0FFFF", label = "Lung L"]
        Lung_R [class= "OAR", color = "#FFDAB9" label = "Lung R"]
        Lung_B [class= "OAR", color = "#DAA532", label = "Lung B"]
        Skin [class= "OAR", color = "#F0FFF0", label = "Skin"]
        }
    subgraph External {cluster = true;
        Body [class= "External", shape = "oval", color = "#00ff00", label = "Body"]
        }
graph [label = "Example Structure Relations\n"]

GTV -- CTV [class = "Equals", style = "tapered", penwidth="5", dir=both]
GTV -- PTV [class = "Contain", style = "dotted",  penwidth=1]
GTV -- eval_PTV [class = "Contain", style = "dotted", penwidth=1]
GTV -- Skin [class = "Island", style = "dotted", headlabel="Embeds", taillabel="Surrounds"]
GTV -- Lung_L [class = "Contain", style = "dotted", penwidth=1]
GTV -- Lung_B [class = "Contain", style = "dotted", penwidth=1]
CTV -- PTV [class = "Contain", style = "tapered", penwidth="5", dir=forward]
#CTV -- eval_PTV [label="Contain"]
CTV -- Skin [class = "Island", style = "dotted", headlabel="Embeds", taillabel="Surrounds"]
CTV -- Lung_L [class = "Contain", style = "dotted",]
#CTV -- Lung_B [label="Contain"]
PTV -- eval_PTV [class = "Group", style = "solid", headlabel="Incorporates", taillabel="Partitions"]
#PTV -- Skin [label="Island"]
PTV -- Lung_L [label="Overlaps", style = "bold"]
#PTV -- Lung_B [label="Overlaps"]
#eval_PTV -- Skin [label="Island"]
eval_PTV -- Lung_L [class = "Group", style = "solid", headlabel="Partitions", taillabel="Incorporates"]
#eval_PTV -- Lung_B [label="Contain"]
Lung_L -- Lung_B [class = "Contain", style = "tapered", penwidth="5", dir=back]
Lung_R -- Lung_B [class = "Contain", style = "tapered", penwidth="5", dir=back]
#Body -- GTV [label="Contain"]
#Body -- CTV [label="Contain"]
#Body -- PTV [label="Contain"]
#Body -- eval_PTV [label="Contain"]
Body -- Skin [class = "Cut-out", style = "dashed",  headlabel="Partitions", taillabel="Incorporates"]
Body -- Lung_L [class = "Contain", style = "tapered", penwidth="5", dir=forward]
Body -- Lung_R [class = "Contain", style = "tapered", penwidth="5", dir=forward]
Body -- Lung_B [class = "Contain", style = "tapered", penwidth="5", dir=forward]
GTV [tooltip = "ID: GTV\nROI: 38\n\nVolumeType: GTV\nDicomType: GTV\nCode: GTVp\nLabel: Primary Gross Tumor Volume\nScheme: 99VMS_STRUCTCODE\nVolume: 8.0\nEq Sp Diam: 2.5\nLength: 2.6\nRange: (-0.4, -3.0)"]
CTV [tooltip = "ID: CTV\nROI: 24\nName: Internal Target Volume\nVolumeType: CTV\nDicomType: CTV\nCode: ITV\nLabel: Internal Target Volume\nScheme: 99VMS_STRUCTCODE\nVolume: 34.4\nEq Sp Diam: 4.0\n\nLength: 3.6\nRange: (0.0, -3.6)"]
PTV [tooltip = "ID: PTV\nROI: 30\nName: PTV defined by Radiation Oncologist\nVolumeType: PTV\nDicomType: PTV\nCode: PTVp\nLabel: Primary Planning Target Volume\nScheme: 99VMS_STRUCTCODE\nVolume: 74.6\nEq Sp Diam: 5.2\n\nLength: 4.6\nRange: (0.6, -4.0)"]
Body [tooltip = "ID: Body\nROI: 1\n\nVolumeType: BODY\nDicomType: EXTERNAL\nCode: BODY\nLabel: Body\nScheme: 99VMS_STRUCTCODE\nVolume: 28,951.6\nEq Sp Diam: 38.1\nRadius: 16.5\nLength: 33.8\nRange: (10.6, -23.2)"]
eval_PTV [tooltip = "ID: eval_PTV\nROI: 41\nName: PTV High Risk for dvh\nVolumeType: PTV\nDicomType: PTV\nCode: PTVp\nLabel: Primary Planning Target Volume\nScheme: 99VMS_STRUCTCODE\nVolume: 630.6\nEq Sp Diam: 10.6\n\nLength: 9.8\nRange: (1.4, -8.4)"]
Lung_L [tooltip = "ID: Lung_L\nROI: 26\nName: Left Lung\nVolumeType: Organ\nDicomType: ORGAN\nCode: 7310\nLabel: Left lung\nScheme: FMA\nVolume: 1,776.0\nEq Sp Diam: 15.0\n\nLength: 24.2\nRange: (5.8, -18.4)"]
Lung_R [tooltip = "ID: Lung_R\nROI: 27\nName: Right Lung\nVolumeType: Organ\nDicomType: ORGAN\nCode: 7309\nLabel: Right lung\nScheme: FMA\nVolume: 2,556.7\nEq Sp Diam: 17.0\n\nLength: 23.8\nRange: (6.6, -17.2)"]
Lung_B [tooltip = "ID: Lung_B\nROI: 25\nName: Both Lungs\nVolumeType: Organ\nDicomType: ORGAN\nCode: 68877\nLabel: Pair of lungs\nScheme: FMA\nVolume: 4,332.7\nEq Sp Diam: 20.2\n\nLength: 25.0\nRange: (6.6, -18.4)"]
Skin [tooltip = "ID: Skin\n\nName: Skin\nVolumeType: Organ\nDicomType: Organ\nCode: 7163\nLabel: Skin\nScheme: FMA\nVolume: 1,072.3\n\n\nLength: 33.8\nRange: (10.6, -23.2)"]
}
```

In [ ]:
structure_set_graph_str = '''
graph StructureSet {
    graph [labelloc = t,
          clusterrank=none,
          bgcolor="#555555",
          fontname = "Helvetica,Arial,sans-serif",
          fontsize = 16,
          fontcolor = "white"
          ]
    node [fixedsize = "shape",
          width = 1,
          height = .6,
          fontname = "Helvetica-Bold",
          fontsize = 12,
          labelloc = "c",
          penwidth = 3,
          style="filled",
          fontcolor = "black"
          ]
    edge [color = "#e27dd6ff",
          penwidth = 3,
          style = "solid",
          arrowhead = "none",
          arrowtail = "none",
          labelfloat=false,
          labelfontname = "Cambria",
          fontcolor = "#55AAFF",
          fontsize = "10"
          ]
    subgraph Targets {
        cluster = true;
        GTV [class = "Target", shape = "pentagon", color = "#ff0000", label = "GTV"]
        CTV [class= "Target", shape = "hexagon", color = "#FFFF00" label = "CTV"]
        PTV [class= "Target", shape = "octagon", color = "#00FFFF", label = "PTV"]
        eval_PTV [class= "Target", shape = "octagon" color = "#00FFFF" label = "eval PTV"]
        }
    subgraph OAR {cluster = true;
        node [shape = "rectangle" style="rounded, filled"]
        Lung_L [class= "OAR", color = "#E0FFFF", label = "Lung L"]
        Lung_R [class= "OAR", color = "#FFDAB9" label = "Lung R"]
        Lung_B [class= "OAR", color = "#DAA532", label = "Lung B"]
        Skin [class= "OAR", color = "#F0FFF0", label = "Skin"]
        }
    subgraph External {cluster = true;
        Body [class= "External", shape = "oval", color = "#00ff00", label = "Body"]
        }
graph [label = "Example Structure Relations\n"]

GTV -- CTV [class = "Equals", style = "tapered", penwidth="5", dir=both]
GTV -- PTV [class = "Contain", style = "dotted",  penwidth=1]
GTV -- eval_PTV [class = "Contain", style = "dotted", penwidth=1]
GTV -- Skin [class = "Island", style = "dotted", headlabel="Embeds", taillabel="Surrounds"]
GTV -- Lung_L [class = "Contain", style = "dotted", penwidth=1]
GTV -- Lung_B [class = "Contain", style = "dotted", penwidth=1]
CTV -- PTV [class = "Contain", style = "tapered", penwidth="5", dir=forward]
#CTV -- eval_PTV [label="Contain"]
CTV -- Skin [class = "Island", style = "dotted", headlabel="Embeds", taillabel="Surrounds"]
CTV -- Lung_L [class = "Contain", style = "dotted",]
#CTV -- Lung_B [label="Contain"]
PTV -- eval_PTV [class = "Group", style = "solid", headlabel="Incorporates", taillabel="Partitions"]
#PTV -- Skin [label="Island"]
PTV -- Lung_L [label="Overlaps", style = "bold"]
#PTV -- Lung_B [label="Overlaps"]
#eval_PTV -- Skin [label="Island"]
eval_PTV -- Lung_L [class = "Group", style = "solid", headlabel="Partitions", taillabel="Incorporates"]
#eval_PTV -- Lung_B [label="Contain"]
Lung_L -- Lung_B [class = "Contain", style = "tapered", penwidth="5", dir=back]
Lung_R -- Lung_B [class = "Contain", style = "tapered", penwidth="5", dir=back]
#Body -- GTV [label="Contain"]
#Body -- CTV [label="Contain"]
#Body -- PTV [label="Contain"]
#Body -- eval_PTV [label="Contain"]
Body -- Skin [class = "Cut-out", style = "dashed",  headlabel="Partitions", taillabel="Incorporates"]
Body -- Lung_L [class = "Contain", style = "tapered", penwidth="5", dir=forward]
Body -- Lung_R [class = "Contain", style = "tapered", penwidth="5", dir=forward]
Body -- Lung_B [class = "Contain", style = "tapered", penwidth="5", dir=forward]
GTV [tooltip = "ID: GTV\r\nROI: 38\r\n\r\nVolumeType: GTV\r\nDicomType: GTV\r\nCode: GTVp\r\nLabel: Primary Gross Tumor Volume\r\nScheme: 99VMS_STRUCTCODE\r\nVolume: 8.0\r\nEq Sp Diam: 2.5\r\nLength: 2.6\r\nRange: (-0.4, -3.0)"]
CTV [tooltip = "ID: CTV\r\nROI: 24\r\nName: Internal Target Volume\r\nVolumeType: CTV\r\nDicomType: CTV\r\nCode: ITV\r\nLabel: Internal Target Volume\r\nScheme: 99VMS_STRUCTCODE\r\nVolume: 34.4\r\nEq Sp Diam: 4.0\r\n\r\nLength: 3.6\r\nRange: (0.0, -3.6)"]
PTV [tooltip = "ID: PTV\r\nROI: 30\r\nName: PTV defined by Radiation Oncologist\r\nVolumeType: PTV\r\nDicomType: PTV\r\nCode: PTVp\r\nLabel: Primary Planning Target Volume\r\nScheme: 99VMS_STRUCTCODE\r\nVolume: 74.6\r\nEq Sp Diam: 5.2\r\n\r\nLength: 4.6\r\nRange: (0.6, -4.0)"]
Body [tooltip = "ID: Body\r\nROI: 1\r\n\r\nVolumeType: BODY\r\nDicomType: EXTERNAL\r\nCode: BODY\r\nLabel: Body\r\nScheme: 99VMS_STRUCTCODE\r\nVolume: 28,951.6\r\nEq Sp Diam: 38.1\r\nRadius: 16.5\r\nLength: 33.8\r\nRange: (10.6, -23.2)"]
eval_PTV [tooltip = "ID: eval_PTV\r\nROI: 41\r\nName: PTV High Risk for dvh\r\nVolumeType: PTV\r\nDicomType: PTV\r\nCode: PTVp\r\nLabel: Primary Planning Target Volume\r\nScheme: 99VMS_STRUCTCODE\r\nVolume: 630.6\r\nEq Sp Diam: 10.6\r\n\r\nLength: 9.8\r\nRange: (1.4, -8.4)"]
Lung_L [tooltip = "ID: Lung_L\r\nROI: 26\r\nName: Left Lung\r\nVolumeType: Organ\r\nDicomType: ORGAN\r\nCode: 7310\r\nLabel: Left lung\r\nScheme: FMA\r\nVolume: 1,776.0\r\nEq Sp Diam: 15.0\r\n\r\nLength: 24.2\r\nRange: (5.8, -18.4)"]
Lung_R [tooltip = "ID: Lung_R\r\nROI: 27\r\nName: Right Lung\r\nVolumeType: Organ\r\nDicomType: ORGAN\r\nCode: 7309\r\nLabel: Right lung\r\nScheme: FMA\r\nVolume: 2,556.7\r\nEq Sp Diam: 17.0\r\n\r\nLength: 23.8\r\nRange: (6.6, -17.2)"]
Lung_B [tooltip = "ID: Lung_B\r\nROI: 25\r\nName: Both Lungs\r\nVolumeType: Organ\r\nDicomType: ORGAN\r\nCode: 68877\r\nLabel: Pair of lungs\r\nScheme: FMA\r\nVolume: 4,332.7\r\nEq Sp Diam: 20.2\r\n\r\nLength: 25.0\r\nRange: (6.6, -18.4)"]
Skin [tooltip = "ID: Skin\r\n\r\nName: Skin\r\nVolumeType: Organ\r\nDicomType: Organ\r\nCode: 7163\r\nLabel: Skin\r\nScheme: FMA\r\nVolume: 1,072.3\r\n\r\n\r\nLength: 33.8\r\nRange: (10.6, -23.2)"]
}
'''
g = pgv.AGraph(structure_set_graph_str)


In [ ]:
g.layout(prog='dot')

In [ ]:
folder=Path.cwd()
name='test_graph'
full_dot_name = name + '.dot'
full_png_name = name + '.png'
full_svg_name = name + '.svg'
dot_path = folder / full_dot_name
png_path = folder / full_png_name
svg_path = folder / full_svg_name

g.draw(png_path)
g.draw(svg_path)
g.write(dot_path)
img = mpimg.imread(png_path)
fig, ax = plt.subplots()
ax.imshow(img)
ax.axis('off')
plt.show()

## Simple Image Browser based on PySimpleGUI

In [ ]:
from PIL import Image, ImageTk
import io

def display_image(filename, window=None)->sg.Window:
    def get_img_data(f, maxsize=(1200, 850), first=False):
        """Generate image data using PIL
        """
        img = Image.open(f)
        img.thumbnail(maxsize)
        if first:                     # tkinter is inactive the first time
            bio = io.BytesIO()
            img.save(bio, format="PNG")
            del img
            return bio.getvalue()
        return ImageTk.PhotoImage(img)

    data=get_img_data(filename)
    if window:
        window['Image'].update(data)
    else:
        image_elem = sg.Image(data=get_img_data(filename, first=True),
                              key='Image')
        window = sg.Window('Image Browser', [[image_elem], [sg.Quit()]])
    # loop until Quit
    while True:
        # read the form
        event, values = window.read()
        # perform button and keyboard operations
        if event == sg.WIN_CLOSED:
            break
        elif event == 'Quit':
            break
    #window.close()
    return window


In [ ]:
# %load -s edge_types,count_edges,plot_multi_edge ./src/multi_graph
def edge_types(graph: nx.Graph)->Tuple[List[Tuple[Any]], List[Tuple[Any]], List[Tuple[Any]]]:
    '''Identify all edge types in a given graph

    Edge types can be Single Edges, Multi-Edges or Loops.

    Args:
        graph (Graph): _description_

    Returns:
        Tuple[EdgeList, EdgeList, EdgeList]: Three different edges list:
                                             A list of Single Edges,
                                             A list of Multi-Edges,
                                             A list of Loops.
    '''
    # Drop the third value (indexer) from each edge tuple
    # The third value is an indexer for multi graphs and is not present in
    # regular graphs.
    edgelist = [(edge[0], edge[1]) for edge in graph.edges]
    edge_count = count_edges(edgelist)
    single_edges = list()
    multi_edges = list()
    loops = list()
    for edge in edgelist:
        if edge[0] == edge[1]:
            loops.append(edge)
        elif edge_count[tuple(sorted(edge))] > 1:
            multi_edges.append(edge)
        else:
            single_edges.append(edge)
    return single_edges, multi_edges, loops

def count_edges(edgelist: List[Tuple[Any]])->Dict[Any, int]:
    '''Count the number of identical edges in an edge list.

    Args:
        edgelist (EdgeList): A list of edges for a Graph or MultiGraph

    Returns:
        Dict[Vertex, int]: A dictionary with the number of edges between two
                           vertices (edge direction is ignored for the count.)
    '''
    sorted_edges = [tuple(sorted(edge)) for edge in edgelist]
    edge_count = Counter(sorted_edges)
    return edge_count

def plot_multi_edge(graph: nx.Graph, pos: Dict[Any, np.ndarray] = None,
                    axis: plt.Axes = None, with_labels=True, font_weight='bold',
                    font_color='white', edge_color='k', arrow='->',
                    directed=True, rad = 0.1,  gap = 10):
    '''Draw a multi-graph with a title.

    This function plots a multi-graph with curved edges where there are
    multiple edges between the same two vertices (including loops).

    Args:
        graph (nx.Graph): A Networkx graph
        pos (VertexPos, optional): The vertex position coordinates to use.
                                   Defaults to None, in which case the
                                   nx.circular_layout is used.
        axis (plt.Axes, optional): The plot box in which to draw the graph.
                                   Defaults to None, in which case a new figure
                                   is generated.
        with_labels (bool, optional): _description_. Defaults to True.
        font_weight (str, optional): _description_. Defaults to 'bold'.
        font_color (str, optional): The color of the text in the plot.
                                     Defaults to 'white'.
        edge_color (str, optional): The color of the edges in the plot.
                                    Defaults to 'k' (black).
        arrow (str, optional): The arrow shape for directed graphs.
                               Defaults to '->'.
        directed (bool, optional): If True, arrows ar added to each edge
                                   indicating direction. Defaults to True.
        rad (float, optional): The radius increment for loops. Defaults to 0.1.
        gap (int, optional): The size of the offset from the center of the
                             vertex to teh start of the edge (accounts for the
                             size of drawn vertices). Defaults to 10.
    '''
    if not pos:
        pos = nx.circular_layout(graph, scale=1.1, center=None, dim=2)
    if not axis:
        axis = plt.subplot()
    if not directed:
        arrow='-'
        draw_arrows = False
    else:
        draw_arrows = True
    plot_params = {
        'edge_color': edge_color,
        'with_labels': with_labels,
        'font_weight': font_weight,
        'font_color': font_color
        }
    edge_props = {
        'arrowstyle': arrow,
        'color': edge_color,
        'shrinkA': 2,
        'shrinkB': gap
        }

    single_edges, multi_edges, loops = edge_types(graph)
    # Plot nodes and single edges
    nx.draw(graph, pos, ax=axis, edgelist=single_edges,
            arrows=draw_arrows, arrowstyle=arrow,
            **plot_params)

    # Draw Multi_edges
    multi_style_tpl='Arc3, rad={rad}'
    edge_count = count_edges(multi_edges)

    # Calculate curvature for the edges
    multi_index = Counter()
    for edge, count in edge_count.items():
        multi_index[edge] = 0.5 - count / 2.0

    # Plot each edge with a different curvature
    for edge in multi_edges:
        sorted_edge = tuple(sorted(edge))
        m_rad = multi_index[sorted_edge] * rad
        multi_index.update([sorted_edge])
        if edge != sorted_edge:
            m_rad = -m_rad  # Deal with different directions
        multi_style = multi_style_tpl.format(rad=m_rad)
        edge_props['connectionstyle'] = multi_style
        start = pos[edge[0]]
        end = pos[edge[1]]
        axis.annotate('', xytext=start, xy=end,
                      in_layout=True, clip_on=False,
                      arrowprops=edge_props)

    # Plot Loops
    # Position of each node in graph
    points = np.array([p for p in pos.values()])
    # Unweighted center of mass for graph
    mid_point = np.array([
        points[:,0].sum() / len(points[:,0]),
        points[:,1].sum() / len(points[:,1])
        ])

    # Count Loops
    # Calculate radius for the loops
    loop_index = Counter(loops)

    for edge in loops:
        # Loop points away from the center of mass
        e_rad = loop_index[edge] * rad
        loop_index.subtract([edge])
        vertex_pt = pos[edge[0]]
        dif = vertex_pt - mid_point
        angle = math.atan2(dif[1], dif[0])
        dx = math.cos(angle) * e_rad / 2
        dy = math.sin(angle) * e_rad / 2
        offset = np.array([dx, dy])
        loop_pt = vertex_pt + offset
        extnt = axis.get_window_extent()
        wh_ratio = extnt.width / extnt.height
        # Create loop
        loop = Arc(loop_pt, e_rad, e_rad*wh_ratio,
                   fill=False, in_layout=True, clip_on=False,
                   edgecolor=plot_params['edge_color'])
        axis.add_patch(loop)
    plt.show()

# %% Done to Here


In [ ]:



adj_matrix_dict = {
    'a': [0, 1, 3, 0, 4],
    'b': [1, 2, 1, 3, 0],
    'c': [3, 1, 1, 0, 1],
    'd': [0, 3, 0, 0, 2],
    'e': [4, 0, 1, 2, 3]
}
adj_matrix = pd.DataFrame(adj_matrix_dict)
adj_matrix.index = adj_matrix.columns
G = from_adjacency_matrix(adj_matrix, directed=False)
box_pos = {
    'a': np.array([0.0, 1.0]),
    'b': np.array([1.0, 1.0]),
    'e': np.array([0.0, 0.0]),
    'c': np.array([0.6, 0.3]),
    'd': np.array([1.0, 0.0])
    }
plot_multi_edge(G, box_pos, directed=False)
